In [1]:
# Define the paths to the images to easily access them later





In [2]:
import os
from astropy.io import fits
from src.functions import *
import numpy as np
from astropy.nddata import CCDData
from astropy.stats import mad_std
import ccdproc as ccdp
from astropy import units as u
import matplotlib.pyplot as plt

In [86]:

def get_collection(path):
    coll = ccdp.ImageFileCollection(path)
    return coll

def make_path(path):
    if not os.path.exists(path): os.system("mkdir -p %s"%path)
    return

def open_ccd_image(file_name, unit):
    im = CCDData(fits.getdata(file_name), meta=fits.getheader(file_name), unit = unit)
    return im

def combine_images(to_calibrate, pars, scale = None):
    pars_forCombining = tuple( [ pars["combine"][key] for key in pars["combine"] ] )
    
    method, sigma_clip_low_thresh, sigma_clip_high_thresh,\
    sigma_clip_func, sigma_clip_dev_func, unit = pars_forCombining
    
    if scale != None:
        # Use a function to scale images
        combined = ccdp.combine(to_calibrate,
                            method=method,
                            scale = scale,
                            sigma_clip=True, 
                            sigma_clip_low_thresh=sigma_clip_low_thresh, 
                            sigma_clip_high_thresh=sigma_clip_high_thresh,
                            sigma_clip_func=sigma_clip_func, 
                            sigma_clip_dev_func=sigma_clip_dev_func,
                            unit = unit)
    else:
        combined = ccdp.combine(to_calibrate,
                            method=method,
                            sigma_clip=True, 
                            sigma_clip_low_thresh=sigma_clip_low_thresh, 
                            sigma_clip_high_thresh=sigma_clip_high_thresh,
                            sigma_clip_func=sigma_clip_func, 
                            sigma_clip_dev_func=sigma_clip_dev_func,
                            unit = unit)
    
    combined.meta['combined'] = True
    return combined


def get_collection(path):
    coll = ccdp.ImageFileCollection(path)
    return coll


def get_files_to_calibrate(path, imagetyp):
    # Obtain a collection with the images
    reduced_images = get_collection(path)
    # Get a filtered list of the images to be calibrated
    to_calibrate = reduced_images.files_filtered(imagetyp = imagetyp, include_path=True)
    return to_calibrate



def calibrate_images(pars):
    # Make sure all paths exist
    make_path(pars["paths"]["comb_out"])
    make_path(pars["paths"]["red_darks"])
    make_path(pars["paths"]["red_flats"])
    make_path(pars["paths"]["red_lights"])
    
    print('-------------- Combining bias --------------')

    # Combine biases into one single image
    calibrated_bias = get_files_to_calibrate(pars["paths"]["Bias"], 'Bias Frame')
    master_bias = combine_images(calibrated_bias, pars)
    master_bias.write(pars["paths"]["comb_out"] + '/combined_bias.fits')
    
    print('-------------- Calibrating darks --------------')
    # Now calibrate darks with bias and combine them into a master image
    calibrated_darks = get_files_to_calibrate(pars["paths"]["Dark"], 'Dark Frame')
    for dark in calibrated_darks:
        name = dark.split("/")[-1]
        print("Calibrating dark: %s and saving it to: %s"%(dark, name))
        
        # -- Extract the proper image array
        im = open_ccd_image(dark, pars["combine"]["unit"])
        
        # Subtract them the bias
        im = ccdp.subtract_bias(im, master_bias)
        
        # Save the result
        im.write(pars["paths"]["red_darks"] + "/" + name)
        
    print('-------------- Combining darks --------------')
    # Having calibrated the darks, we can combine them. We only have one exposure 
    # for the darks, so we are just going to save one dark master image.
    reduced_darks = get_files_to_calibrate(pars["paths"]["red_darks"], 'Dark Frame')
    master_dark = combine_images(reduced_darks, pars)
    master_dark.write(pars["paths"]["comb_out"] + '/combined_dark.fits')
    
    print('-------------- Calibrating Flats --------------')
    # Now it is time to calibrate the flats
    calibrated_flats = get_files_to_calibrate(pars["paths"]["Flat"], 'Flat Frame')
    for flat in calibrated_flats:
        name = flat.split("/")[-1]
        im_flat = open_ccd_image(flat, pars["combine"]["unit"])
        # Subtract the dark current 
        im_flat = ccdp.subtract_dark(im_flat,
                                     master_dark,
                                     exposure_time='exptime',
                                     exposure_unit=u.second)
        im_flat.write(pars["paths"]["red_flats"] + '/' + name)
        
    print('-------------- Combining Flats --------------')
    reduced_flats = get_files_to_calibrate(pars["paths"]["red_flats"], 'Flat Frame')
    
    master_flat = combine_images(reduced_flats, pars, lambda x: 1 / np.median(x) )
    master_flat.write(pars["paths"]["comb_out"] + '/combined_flat.fits')
    
    print('-------------- Calibrating Lights --------------')
    # Now get on with the lights
    calibrated_lights = get_files_to_calibrate(pars["paths"]["Light"], 'Light Frame')
    for light in calibrated_lights:
        name = light.split("/")[-1]
        
        # -- Extract the proper image array
        
        im_light = open_ccd_image(light, pars["combine"]["unit"])

        # Extract the bias
        reduced = ccdp.subtract_bias(im_light, master_bias)

        # Extract the darks
        reduced = ccdp.subtract_dark(reduced, master_dark, 
                                     exposure_time='exptime', exposure_unit=u.second)

        # Correct with the Flats
        reduced = ccdp.flat_correct(reduced, master_flat)
        reduced.write(pars["paths"]["red_lights"] + '/' + name)

    return

In [87]:
print('-------------- Starting calibration --------------')
calibrate_images(pars)
print('-------------- Finished calibration --------------')

-------------- Starting calibration --------------
-------------- Combining bias --------------
-------------- Calibrating darks --------------
Calibrating dark: ./images/SN2021hiz/Dark/Dark_Dark_60_secs_001.fits and saving it to: Dark_Dark_60_secs_001.fits
Calibrating dark: ./images/SN2021hiz/Dark/Dark_Dark_60_secs_002.fits and saving it to: Dark_Dark_60_secs_002.fits
Calibrating dark: ./images/SN2021hiz/Dark/Dark_Dark_60_secs_003.fits and saving it to: Dark_Dark_60_secs_003.fits
Calibrating dark: ./images/SN2021hiz/Dark/Dark_Dark_60_secs_004.fits and saving it to: Dark_Dark_60_secs_004.fits
Calibrating dark: ./images/SN2021hiz/Dark/Dark_Dark_60_secs_005.fits and saving it to: Dark_Dark_60_secs_005.fits
Calibrating dark: ./images/SN2021hiz/Dark/Dark_Dark_60_secs_006.fits and saving it to: Dark_Dark_60_secs_006.fits


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


Calibrating dark: ./images/SN2021hiz/Dark/Dark_Dark_60_secs_007.fits and saving it to: Dark_Dark_60_secs_007.fits
Calibrating dark: ./images/SN2021hiz/Dark/Dark_Dark_60_secs_008.fits and saving it to: Dark_Dark_60_secs_008.fits
Calibrating dark: ./images/SN2021hiz/Dark/Dark_Dark_60_secs_009.fits and saving it to: Dark_Dark_60_secs_009.fits
Calibrating dark: ./images/SN2021hiz/Dark/Dark_Dark_60_secs_010.fits and saving it to: Dark_Dark_60_secs_010.fits
-------------- Combining darks --------------
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu p

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
-------------- Calibrating Flats --------------


...

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader inste

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
INFO:astropy:using the unit 

-------------- Calibrating Lights --------------
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: u